<a href="https://colab.research.google.com/github/NgPcAnhh/study/blob/main/API_Speech2Text%2BAnalyze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi uvicorn nest_asyncio jinja2
!pip install git+https://github.com/openai/whisper.git
!pip install pyngrok
!pip install python-multipart
!pip install transformers torch spacy language-tool-python nltk scikit-learn praat-parselmouth textstat sentence-transformers pyngrok pydantic nest_asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 2.9 MB/s eta 0:00:00
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-w8vnhg1q
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-w8vnhg1q
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [ ]:
import spacy
import language_tool_python
import textstat
import nltk
from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sentence_transformers import SentenceTransformer, util
import language_tool_python
from datetime import datetime


# Load mô hình spaCy để nhận diện Candidate & Examiner
nlp = spacy.load("en_core_web_sm")

# Load BERT để phân loại câu trả lời
classifier = pipeline("text-classification", model="textattack/bert-base-uncased-SST-2")

# Load công cụ kiểm tra ngữ pháp
grammar_tool = language_tool_python.LanguageTool('en-UK')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu

INFO:language_tool_python.download_lt:Unzipping /tmp/tmpn4jkv07t.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-6.4.zip to /root/.cache/language_tool_python.


khởi tạo api

In [ ]:
import tempfile
import whisper
import nest_asyncio
from pyngrok import ngrok, conf
from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from pydantic import BaseModel
from typing import List
import uvicorn
import logging
from datetime import datetime
import re
from sentence_transformers import SentenceTransformer, util
import language_tool_python

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Thiết lập logging với format chi tiết
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Khởi tạo các thành phần cần thiết
NGROK_AUTH_TOKEN = "2sWjZTSTemDeBnksDgYJvraqVku_7aYAKKMyNoNhKzoUxhHFU"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Khởi tạo FastAPI app
app = FastAPI(
    title="Speech Recognition and Processing API",
    description="API for transcribing speech to text using Whisper model and processing the transcription",
    version="1.0.0"
)

# Cấu hình CORS - cho phép cross-origin requests
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["POST", "GET"],
    allow_headers=["*"],
)

# Khởi tạo model Whisper
try:
    model = whisper.load_model("base")
    logger.info("Whisper model loaded successfully")
except Exception as e:
    logger.error(f"Error loading Whisper model: {str(e)}")
    raise

def clean_text(text: str) -> str:
    """
    Văn bản đã được làm sạch
    """
    # Xóa các ký tự đặc biệt
    text = text.replace('"""', '')
    text = text.replace('\n', ' ')
    text = text.replace(')', '')

    # Loại bỏ khoảng trắng thừa
    text = ' '.join(text.split())

    return text

def extract_part1(text: str, conversation: List[str]) -> List[str]:
    # Đánh dấu phần bắt đầu của Part 1
    conversation.append("--- Part 1 ---")

    # Tìm vị trí bắt đầu và kết thúc của Part 1
    intro_mark = text.find("Now, in this first part")
    ending_part1_mark = text.find("That is the end of part 1")

    if intro_mark != -1 and ending_part1_mark != -1:
        part1_text = text[intro_mark:ending_part1_mark]

        # Tách câu dựa trên dấu chấm và dấu hỏi
        sentences = []
        current_sentence = ""

        for char in part1_text:
            current_sentence += char
            if char in ['?', '.']:
                if current_sentence.strip():
                    sentences.append(current_sentence.strip())
                current_sentence = ""

        # Xử lý từng câu trong Part 1
        i = 0
        while i < len(sentences):
            sentence = sentences[i].strip()

            # Nếu là câu hỏi
            if '?' in sentence:
                # Thêm câu hỏi vào conversation
                conversation.append(f"examiner: {sentence}")

                # Tìm câu trả lời (các câu tiếp theo không phải là câu hỏi)
                answer_parts = []
                j = i + 1
                while j < len(sentences) and '?' not in sentences[j]:
                    answer = sentences[j].strip()
                    if answer:
                        answer_parts.append(answer)
                    j += 1

                # Nếu có câu trả lời, thêm vào conversation
                if answer_parts:
                    conversation.append(f"candidate: {' '.join(answer_parts)}")
                i = j - 1
            i += 1

    return conversation

def extract_part2(text: str, conversation: List[str]) -> List[str]:
    # Đánh dấu phần bắt đầu của Part 2
    conversation.append("--- Part 2 ---")

    # Tìm vị trí bắt đầu và kết thúc của Part 2
    part2_start = text.find("Now, let's move on to part 2")
    part2_end = text.find("That is the end of part 2")

    if part2_start != -1 and part2_end != -1:
        part2_text = text[part2_start:part2_end]

        # Tìm câu hỏi chính của Part 2
        question_start = part2_text.find("I'd like you to")
        if question_start != -1:
            main_question = part2_text[question_start:].split(".")[0] + "."
            conversation.append(f"examiner: {main_question}")

        # Tìm phần trả lời của thí sinh
        speaking_start = part2_text.find("Start speaking now")
        if speaking_start != -1:
            answer = part2_text[speaking_start:].strip()
            if answer:
                conversation.append(f"candidate: {answer}")

    return conversation

def extract_part3(text: str, conversation: List[str]) -> List[str]:
    # Đánh dấu phần bắt đầu của Part 3
    conversation.append("--- Part 3 ---")

    # Tìm vị trí bắt đầu của Part 3
    part3_start = text.find("Now let's move on to part 3")
    if part3_start != -1:
        part3_text = text[part3_start:]

        # Tách câu
        sentences = []
        current_sentence = ""

        for char in part3_text:
            current_sentence += char
            if char in ['?', '.']:
                if current_sentence.strip():
                    sentences.append(current_sentence.strip())
                current_sentence = ""

        # Xử lý từng câu trong Part 3
        i = 0
        while i < len(sentences):
            sentence = sentences[i].strip()

            # Nếu là câu hỏi
            if '?' in sentence:
                # Thêm câu hỏi vào conversation
                conversation.append(f"examiner: {sentence}")

                # Tìm câu trả lời
                answer_parts = []
                j = i + 1
                while j < len(sentences) and '?' not in sentences[j]:
                    answer = sentences[j].strip()
                    if answer:
                        answer_parts.append(answer)
                    j += 1

                # Nếu có câu trả lời, thêm vào conversation
                if answer_parts:
                    conversation.append(f"candidate: {' '.join(answer_parts)}")
                i = j - 1
            i += 1

    return conversation

def process_transcription(text: str) -> List[str]:
    try:
        # Khởi tạo list chứa các dòng hội thoại
        conversation = []

        # Làm sạch text
        cleaned_text = clean_text(text)

        # Xử lý từng phần
        conversation = extract_part1(cleaned_text, conversation)
        conversation = extract_part2(cleaned_text, conversation)
        conversation = extract_part3(cleaned_text, conversation)

        return conversation

    except Exception as e:
        logger.error(f"Error in process_transcription: {str(e)}")
        return ["Error processing transcription"]

# Mô tả dữ liệu đầu vào
class TranscriptionInput(BaseModel):
    transcription: list

# Your provided function, unchanged
def parse_transcription(transcription):
    # Tách các phần dựa theo mẫu '--- Part X ---'
    parts = re.split(r'--- Part (\d+) ---', transcription)
    result = []  # List chứa các list con, mỗi list con có dạng: [part, question, response]

    # Bắt đầu từ index 1, duyệt theo cặp: số part và nội dung của part
    for i in range(1, len(parts), 2):
        try:
            part_number = int(parts[i].strip())
        except ValueError:
            continue
        part_text = parts[i + 1].strip()

        # Tìm kiếm các cặp câu hỏi - câu trả lời
        pattern = re.compile(r'examiner: (.*?)\n(candidate: .*?)(?=\nexaminer:|\n--- Part|\Z)', re.DOTALL)
        pairs = pattern.findall(part_text)

        for question, response in pairs:
            # Loại bỏ tiền tố "candidate: " trong câu trả lời
            response_clean = response.replace("candidate: ", "").strip()
            # Append kết quả dưới dạng list con: [part, question, response]
            result.append([part_number, question.strip(), response_clean])

    return result

# Your provided function, modified to return the data instead of printing
def analyze_answer_relevance(data_list):
    try:
        # Initialize the model
        model = SentenceTransformer('all-MiniLM-L6-v2')

        # Result list
        results = []

        def evaluate_relevance(question, answer):
            """Calculate cosine similarity between question and answer embeddings."""
            q_embedding = model.encode(question, convert_to_tensor=True)
            a_embedding = model.encode(answer, convert_to_tensor=True)

            similarity = util.cos_sim(q_embedding, a_embedding).item()
            similarity_percentage = similarity * 100

            if similarity < 0.2:
                label = "Not Relevant"
            elif similarity < 0.5:
                label = "Neutral"
            else:
                label = "Relevant"

            return similarity_percentage, label

        # Analyze each entry
        for entry in data_list:
            part_number = entry[0]
            question = entry[1]
            answer = entry[2]

            # Calculate relevance
            relevance_score, relevance_label = evaluate_relevance(question, answer)

            # Store analysis for this entry
            results.append({
                "part_number": part_number,
                "question": question,
                "answer": answer,
                "relevance_score": relevance_score,
                "label": relevance_label
            })

        return results

    except Exception as e:
        return {"error": str(e)}

def analyze_all_answers(data_list):
    try:
        # Initialize the grammar tool
        grammar_tool = language_tool_python.LanguageTool('en-UK')

        # Dictionary to store all errors
        all_results = {}

        # Analyze each entry in the data list
        for entry in data_list:
            part_number = entry[0]
            question = entry[1]
            answer_text = entry[2]  # Last element is always the answer

            # Get all matches (errors)
            matches = grammar_tool.check(answer_text)

            # Filter out punctuation errors
            excluded_rules = [
                "COMMA_COMPOUND_SENTENCE",
                "COMMA_PARENTHESIS_WHITESPACE",
                "PUNCTUATION_COMMA",
                "PUNCTUATION_PERIOD",
                "PUNCTUATION_QUOTATION",
                "PUNCTUATION_WHITESPACE",
                "COMMA_PARENTHESIS_START_WHITESPACE",
                "COLON_WHITESPACE",
                "EXTRA_COMMA",
                "MISSING_COMMA",
                "SEMICOLON_WHITESPACE",
                "EXTRA_PERIOD",
                "MISSING_PERIOD",
                "EXTRA_WHITESPACE",
                "MISSING_WHITESPACE",
                "WHITESPACE_RULE"
            ]

            # Create markers for error positions
            error_markers = list(" " * len(answer_text))
            errors_found = []

            for match in matches:
                if not match.ruleId.startswith("PUNCT") and match.ruleId not in excluded_rules:
                    # Mark error position
                    for pos in range(match.offset, match.offset + match.errorLength):
                        if pos < len(error_markers):
                            error_markers[pos] = "^"

                    error_info = {
                        "error_type": match.ruleId,
                        "message": match.message,
                        "context": match.context,
                        "suggestion": match.replacements[0] if match.replacements else "No suggestion available",
                        "position": (match.offset, match.offset + match.errorLength),
                        "text": answer_text[match.offset:match.offset + match.errorLength]
                    }
                    errors_found.append(error_info)

            # Store results for this part
            all_results[part_number] = {
                "question": question,
                "answer": answer_text,
                "errors": errors_found
            }

        return all_results

    except Exception as e:
        return {"error": str(e)}

@app.post("/transcribe_and_process")
async def transcribe_and_process(file: UploadFile = File(...)):
    try:
        # Log thông tin request
        logger.info(f"Received file: {file.filename}")
        start_time = datetime.utcnow()

        # Kiểm tra định dạng file
        if not file.filename.lower().endswith(".wav"):
            return JSONResponse(
                status_code=400,
                content={"error": "Invalid file format. Only WAV files are accepted"}
            )

        # Đọc và xử lý file
        audio_bytes = await file.read()

        # Tạo temporary file để xử lý
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=True) as tmp:
            # Ghi dữ liệu vào temporary file
            tmp.write(audio_bytes)
            tmp.flush()

            # Thực hiện transcribe
            result = model.transcribe(tmp.name, fp16=False)
            transcription = result.get("text", "")

            # Xử lý transcription thành conversation
            conversation = process_transcription(transcription)

            # Chuyển đổi conversation thành định dạng chuỗi
            transcription_str = '\n'.join(conversation)
            parsed_transcription = parse_transcription(transcription_str)

            # Phân tích độ liên quan của câu trả lời
            relevance_result = analyze_answer_relevance(parsed_transcription)
            # Phân tích lỗi ngữ pháp
            analysis_result = analyze_all_answers(parsed_transcription)

            # Log thời gian hoàn thành
            end_time = datetime.utcnow()
            processing_time = (end_time - start_time).total_seconds()
            logger.info(f"Processing completed in {processing_time:.2f} seconds")

            # Trả về kết quả
            return JSONResponse(
                content={"conversation": conversation, "relevance": relevance_result, "analysis": analysis_result},
                status_code=200
            )

    except Exception as e:
        logger.error(f"Error processing request: {str(e)}")
        return JSONResponse(
            status_code=500,
            content={"error": "Internal server error during processing"}
        )

@app.get("/health")
async def health_check():
    return {
        "status": "healthy",
        "timestamp": datetime.utcnow().isoformat(),
        "model_loaded": model is not None
    }

def start_server():
    try:
        # Khởi tạo ngrok tunnel
        public_url = ngrok.connect(8000)

        # In đậm thông tin URL trong terminal
        print("\n" + "="*50)
        print("\033[1m\033[92m[Server Information]\033[0m")  # Màu xanh lá, in đậm
        print(f"⏰ Server started at: {datetime.utcnow()}")
        print(f"🌐 Local URL: \033[94mhttp://localhost:8000\033[0m")
        print(f"🚀 Public URL (Ngrok): \033[94m{public_url}\033[0m")
        print(f"📚 API Documentation: \033[94m{public_url}/docs\033[0m")
        print(f"💡 Health Check: \033[94m{public_url}/health\033[0m")
        print("="*50 + "\n")

        # Logging cho file log
        logger.info(f"Server started at: {datetime.utcnow()}")
        logger.info(f"Local URL: http://localhost:8000")
        logger.info(f"Public URL: {public_url}")
        logger.info(f"API Documentation: {public_url}/docs")
        logger.info(f"Health Check: {public_url}/health")

        return public_url

    except Exception as e:
        error_msg = f"Failed to start ngrok tunnel: {str(e)}"
        print(f"\n\033[91m[ERROR] {error_msg}\033[0m")  # Màu đỏ cho lỗi
        logger.error(error_msg)
        raise

if __name__ == "__main__":
    try:
        print("\n\033[1m🚀 Starting Speech Recognition and Processing API Server...\033[0m\n")  # In đậm

        # Khởi động server với ngrok
        public_url = start_server()

        print("\n\033[1m📡 Starting FastAPI Server...\033[0m\n")  # In đậm

        # Chạy FastAPI server
        uvicorn.run(
            app,
            host="0.0.0.0",
            port=8000,
            log_level="info"
        )
    except Exception as e:
        error_msg = f"Server startup error: {str(e)}"
        print(f"\n\033[91m[ERROR] {error_msg}\033[0m")  # Màu đỏ cho lỗi
        logger.error(error_msg)

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-6' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    se


🚀 Starting Speech Recognition and Processing API Server...


[Server Information]
⏰ Server started at: 2025-02-11 15:03:20.479024
🌐 Local URL: http://localhost:8000
🚀 Public URL (Ngrok): NgrokTunnel: "https://e493-35-243-188-201.ngrok-free.app" -> "http://localhost:8000"
📚 API Documentation: NgrokTunnel: "https://e493-35-243-188-201.ngrok-free.app" -> "http://localhost:8000"/docs
💡 Health Check: NgrokTunnel: "https://e493-35-243-188-201.ngrok-free.app" -> "http://localhost:8000"/health


📡 Starting FastAPI Server...



INFO:     Started server process [5129]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     42.116.198.240:0 - "POST /transcribe_and_process HTTP/1.1" 200 OK
INFO:     42.116.198.240:0 - "POST /transcribe_and_process HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [5129]
